In [1]:
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
import random

## Widgets initialization

In [2]:
line_out = widgets.HTML()
current_out = widgets.HTML()
finish_out = widgets.HTML()

step_btn = widgets.Button(description='Step', button_style='primary')
gen_btn = widgets.Button(description='Generate line', button_style='primary')
finish_btn = widgets.Button(description='Step all', button_style='danger')

## TuringMachine class

In [3]:
class Tape(object):
    
    blank_symbol = "#"
    
    def __init__(self,
                 tape_string = ""):
        self.__tape = dict((enumerate(tape_string)))
        # last line is equivalent to the following three lines:
        #self.__tape = {}
        #for i in range(len(tape_string)):
        #    self.__tape[i] = input[i]
        
    def __str__(self):
        s = ""
        min_used_index = min(self.__tape.keys()) 
        max_used_index = max(self.__tape.keys())
        for i in range(min_used_index, max_used_index + 1):
            s += self.__tape[i]
        return s    
   
    def __getitem__(self,index):
        if index in self.__tape:
            return self.__tape[index]
        else:
            return Tape.blank_symbol

    def __setitem__(self, pos, char):
        self.__tape[pos] = char 

        
class TuringMachine(object):
    
    def __init__(self, 
                 tape = "", 
                 blank_symbol = "#",
                 initial_state = "",
                 transition_function = None):
        self.__tape = Tape(tape)
        self.__head_position = 1
        self.__blank_symbol = blank_symbol
        self.__current_state = initial_state
        if transition_function == None:
            self.__transition_function = {}
        else:
            self.__transition_function = transition_function
        
    def get_tape(self): 
        return str(self.__tape)
    
    def step(self):
        char_under_head = self.__tape[self.__head_position]
        x = (self.__current_state, char_under_head)
        if x in self.__transition_function:
            y = self.__transition_function[x]
            self.__tape[self.__head_position] = y[1]
            if y[2] == "r":
                self.__head_position += 1
            elif y[2] == "l":
                self.__head_position -= 1
            self.__current_state = y[0]
            current_out.value = x[0] + ' ' + x[1] + ' -> ' + y[0] + ' ' + y[1] + ' ' + y[2]
            return False
        else:
            return True

## Example problem

In [4]:
def gen_line(l=7):
    line = [random.choice('AB') for i in range(l)]
    return ''.join(['#'] + line + ['#'])

In [5]:
transition_function = {
    ('q0', 'A'): ('q0', 'A', 'r'),
    ('q0', 'B'): ('q0', '*', 'r'),
    ('q0', '*'): ('q0', '*', 'r'),
    ('q0', '#'): ('q1', '#', 'l'),
    
    ('q1', 'A'): ('q2', 'A', 'l'),
    ('q1', '*'): ('q1', '#', 'l'),
    ('q1', '#'): ('q1', '#', 'r'),
    
    ('q2', 'A'): ('q3', '*', 'r'),
    ('q2', '*'): ('q2', '*', 'l'),
    ('q2', '#'): ('q5', '#', 'r'),
    
    ('q3', 'A'): ('q4', 'A', 'l'),
    ('q3', '*'): ('q3', '*', 'r'),
    
    ('q4', '*'): ('q2', 'A', 'l'),
    
    ('q5', '*'): ('q5', '#', 'r')
}

## Event listeners

In [6]:
line = None
t = None
is_final = False

In [7]:
def on_gen_btn(d):
    global line, t, is_final

    is_final = False
    line = gen_line(5)
    t = TuringMachine(line, 
                      initial_state = 'q0',
                      transition_function=transition_function)
    line_out.value = t.get_tape()
    current_out.value = ' '
    finish_out.value = 'In progress'

def on_step_btn(d):
    global is_final
    if not is_final:
        is_final = t.step()
        line_out.value = t.get_tape()
    else:
        finish_out.value = 'Finish'
        
def on_finish_btn(d):
    global is_final
    while not is_final:
        is_final = t.step()
    line_out.value = t.get_tape() 
    finish_out.value = 'Finish'

In [8]:
gen_btn.on_click(on_gen_btn)
step_btn.on_click(on_step_btn)
finish_btn.on_click(on_finish_btn)
on_gen_btn(None)

## Final result

### Simple

In [9]:
display(gen_btn, step_btn, finish_btn)

Button(button_style='primary', description='Generate line', style=ButtonStyle())

Button(button_style='primary', description='Step', style=ButtonStyle())

Button(button_style='danger', description='Step all', style=ButtonStyle())

In [11]:
display(line_out)
display(current_out)
display(finish_out)

HTML(value='#*ABBB#')

HTML(value='q0 B -> q0 * r')

HTML(value='In progress')

### A more advanced

In [12]:
from ipywidgets import Layout, Box, Label

In [13]:
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_items = [
    Box([Label(value='Current line value'), line_out], layout=form_item_layout),
    Box([Label(value='Current transition'), current_out], layout=form_item_layout),
    Box([Label(value='Finish status'), finish_out], layout=form_item_layout),
    Box([gen_btn, step_btn, finish_btn], layout=form_item_layout)
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))

In [14]:
on_gen_btn(None)

In [15]:
form

Box(children=(Box(children=(Label(value='Current line value'), HTML(value='#ABABB#')), layout=Layout(display='flex', flex_flow='row', justify_content='space-between')), Box(children=(Label(value='Current transition'), HTML(value=' ')), layout=Layout(display='flex', flex_flow='row', justify_content='space-between')), Box(children=(Label(value='Finish status'), HTML(value='In progress')), layout=Layout(display='flex', flex_flow='row', justify_content='space-between')), Box(children=(Button(button_style='primary', description='Generate line', style=ButtonStyle()), Button(button_style='primary', description='Step', style=ButtonStyle()), Button(button_style='danger', description='Step all', style=ButtonStyle())), layout=Layout(display='flex', flex_flow='row', justify_content='space-between'))), layout=Layout(align_items='stretch', border='solid 2px', display='flex', flex_flow='column', width='50%'))